<a href="https://colab.research.google.com/github/miguelsmachado/Aproching_any_ML_probloem/blob/main/Banco_de_dados_Magic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [135]:
import pandas as pd


In [164]:
from urllib.request import urlopen

url = 'https://gatherer.wizards.com/Pages/Card/Details.aspx?printed=false&multiverseid=385786'

response = urlopen(url)
html = response.read()

In [165]:
html = html.decode('utf-8')

In [166]:
def trata_html(input):
    return " ".join(input.split()).replace('> <', '><')

In [167]:
html = trata_html(html)

In [117]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

# Retirando informações da carta

In [118]:
itens = soup.findAll('div', class_="label")
i = 0
index = []
caracteristicas = []
for item in itens:
  index.append((item.get_text().lstrip(), i))
  caracteristicas.append((item.get_text().lstrip()))
  i += 1
caracteristicas

['Community Rating:',
 'Card Name:',
 'Mana Cost:',
 'Converted Mana Cost:',
 'Types:',
 'Card Text:',
 'Flavor Text:',
 'Expansion:',
 'Rarity:',
 'All Sets:',
 'Card Number:',
 'Artist:',
 '']

## Nome da carta

In [119]:
if "Card Name:" in caracteristicas:
  card_name = itens[caracteristicas.index("Card Name:")].find_next().get_text().lstrip()
card_name

'Encrust'

## Custo de mana

In [120]:
mana_cost = []
for mana in soup.findAll('img', {"align":"absbottom"}):
  mana_cost.append(mana.get("alt"))
mana_cost

['1', 'Blue', 'Blue']

## Tipo de carta

In [121]:
if "Types:" in caracteristicas:
  types = itens[caracteristicas.index("Types:")].find_next().get_text().lstrip()
types

'Enchantment — Aura'

## Texto da carta

In [122]:
text = []
for phrase in soup.findAll('div', class_="cardtextbox"):
  text.append(phrase.get_text())
text = "\n".join(text)
print(text)

Enchant artifact or creature
Enchanted permanent doesn't untap during its controller's untap step and its activated abilities can't be activated.


# Poder

In [125]:
if "P/T:" in caracteristicas:
  power = itens[caracteristicas.index("P/T:")].find_next().find_next().get_text().lstrip()
else:
  power = "0 / 0"
atack = int(power.split("/")[0])
print(atack)
defense = int(power.split("/")[1])
print(defense)

0
0


## Edição

In [126]:
if "Expansion:" in caracteristicas:
  edition = itens[caracteristicas.index("Expansion:")].find_next().get_text().lstrip()
edition

'Magic 2015 Core Set'

## Raridade

In [127]:
if "Rarity:" in caracteristicas:
  rarity = itens[caracteristicas.index("Rarity:")].find_next().get_text().lstrip()
rarity

'Common'

## Imagem

In [128]:
image = soup.find('div', class_="cardImage").find('img')
image_link = "https://gatherer.wizards.com" + image.get('src')[5:]
print(image_link)

https://gatherer.wizards.com/Handlers/Image.ashx?multiverseid=385786&type=card


## Gerando o Card

In [134]:
card = {"card_name": card_name,
        "types": types,
        "mana_cost": mana_cost,
        "text": text,
        "atack": atack,
        "defense": defense,
        "edition": edition,
        "rarity": rarity,
        "image": image_link
}

card

{'atack': 0,
 'card_name': 'Encrust',
 'defense': 0,
 'edition': 'Magic 2015 Core Set',
 'image': 'https://gatherer.wizards.com/Handlers/Image.ashx?multiverseid=385786&type=card',
 'mana_cost': ['1', 'Blue', 'Blue'],
 'rarity': 'Common',
 'text': "Enchant artifact or creature\nEnchanted permanent doesn't untap during its controller's untap step and its activated abilities can't be activated.",
 'types': 'Enchantment — Aura'}

In [138]:
pd.DataFrame.from_dict(card, orient='index').T

,card_name,types,mana_cost,text,atack,defense,edition,rarity,image
0,Encrust,Enchantment — Aura,"[1, Blue, Blue]",Enchant artifact or creature\nEnchanted perman...,0,0,Magic 2015 Core Set,Common,https://gatherer.wizards.com/Handlers/Image.as...


# Criando uma rotina de scraping

In [159]:
from urllib.request import urlopen
import pandas as pd
from bs4 import BeautifulSoup
import time

def trata_html(input):
    return " ".join(input.split()).replace('> <', '><')

cards = []

inicio = time.time()
for i in range(1,101):
  card = {}

  url = 'https://gatherer.wizards.com/Pages/Card/Details.aspx?printed=false&multiverseid=' +str(i)

  response = urlopen(url)
  html = response.read()

  html = html.decode('utf-8')
  html = trata_html(html)

  soup = BeautifulSoup(html, 'html.parser')

  # Verificando a possilidade da página não existir
  if soup.title.get_text() == " Gatherer - Magic: The Gathering ":
    continue

  # Obtendo as infomações gerais
  itens = soup.findAll('div', class_="label")
  i = 0
  index = []
  caracteristicas = []
  for item in itens:
    index.append((item.get_text().lstrip(), i))
    caracteristicas.append((item.get_text().lstrip()))
    i += 1
  
  if "Card Name:" in caracteristicas:
    card_name = itens[caracteristicas.index("Card Name:")].find_next().get_text().lstrip()
  
  mana_cost = []
  for mana in soup.findAll('img', {"align":"absbottom"}):
    mana_cost.append(mana.get("alt"))

  if "Types:" in caracteristicas:
    types = itens[caracteristicas.index("Types:")].find_next().get_text().lstrip()

  text = []
  for phrase in soup.findAll('div', class_="cardtextbox"):
    text.append(phrase.get_text())
  text = "\n".join(text)
  
  if "P/T:" in caracteristicas:
    power = itens[caracteristicas.index("P/T:")].find_next().find_next().get_text().lstrip()
  else:
    power = "0 / 0"
  atack = power.split("/")[0]
  defense = power.split("/")[1]

  if "Expansion:" in caracteristicas:
    edition = itens[caracteristicas.index("Expansion:")].find_next().get_text().lstrip()
  
  if "Rarity:" in caracteristicas:
    rarity = itens[caracteristicas.index("Rarity:")].find_next().get_text().lstrip()
  
  image = soup.find('div', class_="cardImage").find('img')
  image_link = "https://gatherer.wizards.com" + image.get('src')[5:]

  card = {"card_name": card_name,
        "types": types,
        "mana_cost": mana_cost,
        "text": text,
        "atack": atack,
        "defense": defense,
        "edition": edition,
        "rarity": rarity,
        "image": image_link}

  cards.append(card)

df = pd.DataFrame(cards)

fim = time.time()
print(f' Tempo de execução: {fim - inicio} segundos')

 Tempo de execução: 110.03138494491577 segundos


In [160]:
df

,card_name,types,mana_cost,text,atack,defense,edition,rarity,image
0,Ankh of Mishra,Artifact,[2],"Whenever a land enters the battlefield, Ankh o...",0,0,Limited Edition Alpha,Rare,https://gatherer.wizards.com/Handlers/Image.as...
1,Basalt Monolith,Artifact,"[3, Tap, Colorless, Colorless, Colorless, 3]",Basalt Monolith doesn't untap during your unta...,0,0,Limited Edition Alpha,Uncommon,https://gatherer.wizards.com/Handlers/Image.as...
2,Black Lotus,Artifact,"[0, Tap]",", Sacrifice Black Lotus: Add three mana of any...",0,0,Limited Edition Alpha,Rare,https://gatherer.wizards.com/Handlers/Image.as...
3,Black Vise,Artifact,[1],"As Black Vise enters the battlefield, choose a...",0,0,Limited Edition Alpha,Uncommon,https://gatherer.wizards.com/Handlers/Image.as...
4,Celestial Prism,Artifact,"[3, 2, Tap]",", : Add one mana of any color.",0,0,Limited Edition Alpha,Uncommon,https://gatherer.wizards.com/Handlers/Image.as...
...,...,...,...,...,...,...,...,...,...
95,Animate Artifact,Enchantment — Aura,"[3, Blue, Tap]",Enchant artifact\nAs long as enchanted artifac...,0,0,Limited Edition Alpha,Uncommon,https://gatherer.wizards.com/Handlers/Image.as...
96,Blue Elemental Blast,Instant,[Blue],Choose one —\n• Counter target red spell.\n• D...,0,0,Limited Edition Alpha,Common,https://gatherer.wizards.com/Handlers/Image.as...
97,Braingeyser,Sorcery,"[Variable Colorless, Blue, Blue]",Target player draws X cards.,0,0,Limited Edition Alpha,Rare,https://gatherer.wizards.com/Handlers/Image.as...
98,Clone,Creature — Shapeshifter,"[3, Blue, Variable Colorless]",You may have Clone enter the battlefield as a ...,0,0,Limited Edition Alpha,Uncommon,https://gatherer.wizards.com/Handlers/Image.as...


In [161]:
df.to_csv("magic_dataset.csv", sep=';', index=False, encoding='utf-8-sig')